In [1]:
import numpy as np
import torch.nn as nn
import torch

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T

import torch.nn.functional as F
from torch.utils.data import sampler
import matplotlib.pyplot as plt
import torchvision.models as models

from tensorboardX import SummaryWriter
dtype = torch.float32

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('using device:', device)

ModuleNotFoundError: No module named 'tensorboardX'

In [ ]:
transform = T.Compose([
    T.Resize(224),
#     T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def getData(batch_size):
    cifar10_train = dset.CIFAR10('../datasets/', train=True, download=True, transform=transform)
    loader_train = DataLoader(cifar10_train, batch_size=batch_size, num_workers=4)

    cifar10_val = dset.CIFAR10('../datasets/', train=False, download=True, transform=transform)
    loader_val = DataLoader(cifar10_val, batch_size=batch_size, num_workers=4)
    
    return (loader_train, loader_val)

def flatten(x):
    N = x.shape[0]
    return x.view(N, -1)

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

def getModel():
    return models.vgg16()
#     return models.resnet18()
def setOptim(model, learning_rate, momentum):
    return optim.SGD(model.parameters(), lr=learning_rate, momentum = momentum)

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')

    num_samples = 0
    num_correct = 0
    model.eval()
    for x, y in loader:
        x = x.to(device=device)
        y = y.to(device=device)
        scores = model(x)
        _, preds = torch.max(scores, dim=1)
        num_correct += (preds == y).sum()
        num_samples += x.shape[0]
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)\n' % (num_correct, num_samples, 100 * acc))
    return acc

In [ ]:
def train(lr, bs, mo, epochs=100):
    print("lr:",lr, " bs:", bs, " mo:", mo)

    loader_train, loader_val = getData(bs)

    model = getModel()
    optimizer = setOptim(model, learning_rate=lr, momentum=mo)

    model = model.to(device=device)

    writer = SummaryWriter(comment="1_epoch")
    train_len = len(loader_train)

    images, labels = next(iter(loader_train))
    
    for e in range(epochs):
        epoch_loss = 0
        epoch_count = 0
        for t, (x, y) in enumerate(loader_train):
            x = x.to(device=device)
            y = y.to(device=device)
            optimizer.zero_grad() # 梯度归零
            model.train()
            scores = model(x)
            loss = F.cross_entropy(scores, y)  # 计算loss
            loss.backward() #　反向传播
            optimizer.step() # 更新参数

            epoch_count += 1
            epoch_loss += loss.item()
            print('Iteration %d, loss = %.4f' %(t, loss.item()))
        writer.add_scalar('train', epoch_loss/epoch_count, e)

        print('epoch %d, loss = %.4f' %(e, epoch_loss/epoch_count))

        acc = check_accuracy(loader_val, model)

        if e % 20 == 0:
            lr = lr/3.0
            optimizer = setOptim(model, learning_rate=lr, momentum=mo)

In [ ]:
learning_rates = [0.01, 0.003]
batch_sizes = [64, 128, 256]
# momentum = [0.9, 0.95, 0.99]

train(0.001, 64, 0.9)

for lr in learning_rates:
    for bs in batch_sizes:
#         for mo in momentum:
        train(lr, bs, mo)